<a href="https://colab.research.google.com/github/samyarsworld/Language-classification-RNN/blob/main/Language_classification_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Detection

### 1. Download the project repository:

Repository address: https://github.com/samyarsworld/Language-classification-RNN/tree/main

The directory includes the data directory, a python utils file containing helper functions, and Python version of the code.

In [ ]:
!git clone https://github.com/samyarsworld/Language-classification-RNN
%cd Language-classification-RNN

Cloning into 'Language-classification-RNN'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 43 (delta 4), reused 13 (delta 2), pack-reused 0
Receiving objects: 100% (43/43), 66.73 KiB | 6.67 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/Language-classification-RNN


### 2. Install the necessary libraries:
Make sure Python (recommended 3.10) and pip (recommended 23.3.1) are installed.

In [ ]:
!pip install -r requirements.txt

### 3. Import Required Libraries and Functions:

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

from utils import load_data, letter_to_tensor, sentence_to_tensor, LETTERS, get_random_sample
from pathlib import Path

### 4. Setup Initial Configurations:

In [ ]:
# Load the data
languages, language_names = load_data()

MODEL_PATH = Path("./model.pt")
N_HIDDEN = 128  # Number of hidden units
EPOCHS = 300000
STEPS = 1000

### 5. Create RNN Model:

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size) -> None:
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(in_features=input_size + hidden_size, out_features=hidden_size)
        self.i2o = nn.Linear(in_features=input_size + hidden_size, out_features=output_size)
        self.softmax = nn.LogSoftmax(dim=1)


    def forward(self, x: torch.Tensor, h: torch.Tensor) -> torch.Tensor:
        combined_tensor = torch.cat((x, h), dim=1)
        return self.softmax(self.i2o(combined_tensor)), self.i2h(combined_tensor)

In [ ]:
# Initialize Model
RNN_model = RNN(len(LETTERS), N_HIDDEN, len(languages))

### 6. Trainin Setup:

In [ ]:
# Optimizer
LR = 0.005
optimizer = torch.optim.SGD(params=RNN_model.parameters(), lr=LR)

# Loss Function
criterion = nn.NLLLoss()
current_loss = 0.0
all_losses = []

In [ ]:
# Training setup
def train(name_tensor, language_tensor):
    next_hidden = RNN_model.init_hidden()

    for i in range(name_tensor.size()[0]):
        output, next_hidden = RNN_model(name_tensor[i], next_hidden)

    loss = criterion(output, language_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return output, loss.item()

### 7. Train the Model:

In [ ]:
# Load model if exists
if MODEL_PATH.exists():
  RNN_model.load_state_dict(torch.load(MODEL_PATH))
  assert("Model was SUCCESSFULLY loaded.")
else:
  assert("Model does NOT exist.")


In [ ]:
# Run traning if the model doesn't exist or needs more training
for epoch in range(EPOCHS):
        language, language_tensor, name,  name_tensor = get_random_sample(languages, language_names)
        output, loss = train(name_tensor, language_tensor)
        current_loss += loss

        if epoch % STEPS == 0:
            all_losses.append(current_loss / STEPS)
            current_loss = 0

            prediction = languages[torch.argmax(output).item()]
            correct = "CORRECT" if prediction == language else f"WRONG -> {language}"
            print(f"{epoch} {epoch / EPOCHS * 100:.0f} {loss:.5f} {name} / {prediction} {correct}")

    # Save the model to local disk
    torch.save(RNN_model.state_dict(), MODEL_PATH)

### 8. Plot Training Losses:

In [ ]:
# Plot losses per 1000 words
plt.plot(all_losses[1:-1], label='Loss', marker='o', linestyle='-')

# Adding titles and labels
plt.title('Plot Title')
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')

# Adding a legend
plt.legend()

# Display the plot
plt.show()

### 9. Perform Predictions:

In [ ]:
def predict(sentence):
    print(f"\n> {sentence}")
    with torch.inference_mode():
        name_tensor = sentence_to_tensor(sentence)

        next_hidden = RNN_model.init_hidden()

        for i in range(name_tensor.size()[0]):
            output, next_hidden = RNN_model(name_tensor[i], next_hidden)

        prediction = languages[torch.argmax(output).item()]
        print(prediction)

# Prediction state
while True:
    sentence = input("Input:")
    if sentence == "q":
        break
    else:
        predict(sentence)